In [10]:
file_path='/Users/alex.wu/java_projects/java-code-parser/mutation_info.jsonl'

In [11]:
import json
from collections import Counter

# 初始化一个 Counter 对象来统计 mutationStrategy
mutation_strategy_counter = Counter()
total_count = 0

with open(file_path, 'r') as file:
    for line in file:
        try:
            json_line = json.loads(line)
            # 提取 mutationStrategy 字段并更新计数器
            mutation_strategy = json_line.get("mutationStrategy", "Unknown")
            mutation_strategy_counter[mutation_strategy] += 1
            total_count += 1
        except json.JSONDecodeError as e:
            print(f"Error parsing line: {e}")

# 打印统计结果和占比
for strategy, count in mutation_strategy_counter.items():
    proportion = (count / total_count) * 100
    print(f"{strategy}: {count}, 占比: {proportion:.2f}%")


Error parsing line: Unterminated string starting at: line 1 column 89 (char 88)
RANDOM_BLOCK_STATEMENT_INFILL_MUTATION: 9863, 占比: 24.09%
RANDOM_BLOCK_STATEMENT_DELETION_MUTATION: 9785, 占比: 23.90%
COMPLETE_CONDITIONAL_BRANCH_DELETION_MUTATION: 4094, 占比: 10.00%
RETURN_EXPRESSION_MUTATION: 3833, 占比: 9.36%
CONDITIONAL_EXPRESSION_DELETION_MUTATION: 6668, 占比: 16.28%
RELATIONAL_EXPRESSION_MUTATION: 2558, 占比: 6.25%
RANDOM_CONDITION_RESERVATION_MUTATION: 2820, 占比: 6.89%
LOOP_CONTROL_DELETION_MUTATION: 992, 占比: 2.42%
SWITCH_RANDOM_BRANCH_DELETION_MUTATION: 336, 占比: 0.82%


In [ ]:
import json
import pandas as pd
from IPython.display import HTML

# 初始化一个列表来存储 JSONL 文件中的数据
data = []

# 读取文件的前1000行
with open(file_path, 'r') as file:
    for i, line in enumerate(file):
        if i < 1000:  # 仅处理前1000行
            try:
                json_line = json.loads(line)
                data.append(json_line)
            except json.JSONDecodeError as e:
                print(f"Error parsing line {i + 1}: {e}")
        else:
            break

for i in data:
    print('------------------------')
    print('filepath: ', i['filePath'])
    print('strategy: ', i['mutationStrategy'])
    print(i['methodPreContext'])
    print('=======')
    print(i['buggyCode'])
    print('=======')
    print(i['originalCode'])
    print('=======')
    print(i['methodPostContext'])


In [ ]:
import json
import pandas as pd

# 初始化一个列表来存储解析后的数据
data = []

# 读取并解析 JSONL 文件
with open(file_path, 'r') as file:
    for line in file:
        try:
            json_line = json.loads(line)
            data.append(json_line)
        except json.JSONDecodeError as e:
            print(f"Error parsing line: {e}")

# 创建 DataFrame
df = pd.DataFrame(data)

In [23]:
import pandas as pd
import re

# 假设 df 是您的原始 DataFrame
# 定义匹配函数头的正则表达式
pattern = r'\b(?:public|protected|private|static|\s)*[\w<>\[\]]+\s+(\w+)\s*\(.*?\)'

def contains_test_method(method_pre_context):
    if pd.isna(method_pre_context):
        return False
    matches = re.finditer(pattern, method_pre_context, re.MULTILINE)
    for match in matches:
        if 'test' in match.group(1).lower():
            return False
    return True

# 应用过滤条件
filtered_df = df[df['methodPreContext'].apply(contains_test_method)]

# 计算过滤的数量
filtered_count = len(df) - len(filtered_df)

print(f"过滤的样本数量是: {filtered_count}")

过滤的样本数量是: 10312


In [ ]:
import pandas as pd
import numpy as np

# 假设 filtered_df 是之前过滤后的 DataFrame

# 计算每个样本的总长度
lengths = filtered_df.apply(lambda row: len(str(row['methodPreContext'])) + len(str(row['originalCode'])) + len(str(row['methodPostContext'])), axis=1)

# 确定保留 80% 样本的长度阈值
threshold = np.percentile(lengths, 80)

# 应用过滤条件
final_df = filtered_df[lengths <= threshold]

# 计算过滤的数量
filtered_count = len(filtered_df) - len(final_df)

print(f"过滤的样本数量是: {filtered_count}")
print(f"过滤的长度阈值是: {threshold}")

In [35]:
final_df.loc[40948]['originalCode']

'if (connection != null) {\n    return Optional.of(this);\n} else {\n    return Optional.empty();\n}'

In [ ]:
# 计算每个样本的 originalCode 字段的长度
code_lengths = final_df['originalCode'].apply(lambda x: len(str(x)))

# 确定保留 80% 样本的长度阈值
code_threshold = np.percentile(code_lengths, 80)

# 应用过滤条件
new_final_df = final_df[code_lengths <= code_threshold]

# 计算过滤的数量
filtered_count = len(final_df) - len(new_final_df)

print(f"过滤的样本数量是: {filtered_count}")
print(f"过滤的 originalCode 长度阈值是: {code_threshold}")

# 打印新 DataFrame 的信息（如果需要）
new_final_df

In [ ]:

# 定义匹配静态方法调用的正则表达式
static_call_pattern = r'\b[A-Z][\w]*\.[a-zA-Z0-9_]+\(\)'

def contains_static_call(row):
    combined_text = f"{row['methodPreContext']} {row['originalCode']} {row['methodPostContext']}"
    return re.search(static_call_pattern, combined_text) is not None

# 应用过滤条件
final_df_no_static_calls = new_final_df[~new_final_df.apply(contains_static_call, axis=1)]

# 计算过滤的数量
filtered_count = len(new_final_df) - len(final_df_no_static_calls)

print(f"过滤的样本数量是: {filtered_count}")

# 打印新 DataFrame 的信息（如果需要）
print(final_df_no_static_calls)


In [43]:

# 统计每种 mutationStrategy 的出现次数
strategy_counts = final_df_no_static_calls['mutationStrategy'].value_counts()

# 计算总样本数
total_count = len(final_df_no_static_calls)

# 打印每种策略及其占比
print("Mutation Strategy 占比:")
for strategy, count in strategy_counts.items():
    proportion = (count / total_count) * 100
    print(f"{strategy}: {count}, 占比: {proportion:.2f}%")

Mutation Strategy 占比:
RANDOM_BLOCK_STATEMENT_INFILL_MUTATION: 3556, 占比: 19.85%
RETURN_EXPRESSION_MUTATION: 3353, 占比: 18.72%
CONDITIONAL_EXPRESSION_DELETION_MUTATION: 2971, 占比: 16.58%
COMPLETE_CONDITIONAL_BRANCH_DELETION_MUTATION: 2718, 占比: 15.17%
RANDOM_BLOCK_STATEMENT_DELETION_MUTATION: 1901, 占比: 10.61%
RELATIONAL_EXPRESSION_MUTATION: 1660, 占比: 9.27%
RANDOM_CONDITION_RESERVATION_MUTATION: 1404, 占比: 7.84%
LOOP_CONTROL_DELETION_MUTATION: 232, 占比: 1.29%
SWITCH_RANDOM_BRANCH_DELETION_MUTATION: 121, 占比: 0.68%


In [44]:
import pandas as pd
import re

# 假设 final_df_no_static_calls 是上一步过滤后的 DataFrame

# 定义匹配链式方法调用的正则表达式，排除 .stream().map() 这样的调用
chain_call_pattern = r'\b\w+\.\w+\(\)(?!\.stream\(\)\.\w+\(\))(\.\w+\(\))*'

def contains_chain_call(row):
    combined_text = f"{row['originalCode']}"
    return re.search(chain_call_pattern, combined_text) is not None

# 应用过滤条件
final_df_no_chain_calls = final_df_no_static_calls[~final_df_no_static_calls.apply(contains_chain_call, axis=1)]

# 计算过滤的数量
filtered_count = len(final_df_no_static_calls) - len(final_df_no_chain_calls)

print(f"过滤的样本数量是: {filtered_count}")

# 打印新 DataFrame 的信息（如果需要）
final_df_no_chain_calls


过滤的样本数量是: 5647


,endLine,mutationStrategy,filePath,startLine,originalCode,buggyCode,methodPreContext,projectName,methodPostContext
5,83,COMPLETE_CONDITIONAL_BRANCH_DELETION_MUTATION,/Users/alex.wu/IdeaProjects/flink/flink-rpc/fl...,81,if (o == this) {\n return 0;\n},,@Override\n public int compareTo(@Nonnu...,flink,\n // tie breaking for ScheduledFutureA...
6,143,RETURN_EXPRESSION_MUTATION,/Users/alex.wu/IdeaProjects/flink/flink-rpc/fl...,143,return tieBreakerUid == that.tieBreakerUid && ...,return tieBreakerUid > that.tieBreakerUid || s...,@Override\n public boolean equals(Objec...,flink,}\n
7,94,RANDOM_BLOCK_STATEMENT_INFILL_MUTATION,/Users/alex.wu/IdeaProjects/flink/flink-rpc/fl...,89,final CompletableFuture<T> guardedFuture = new...,,public static <T> CompletableFuture<T> gua...,flink,return guardedFuture;\n }\n
11,281,CONDITIONAL_EXPRESSION_DELETION_MUTATION,/Users/alex.wu/IdeaProjects/flink/flink-rpc/fl...,279,if (i < lastArgumentIdx) {\n incompleteMeth...,"incompleteMethod.append("": "").append(args[i]);",private StringBuilder getIncompleteMet...,flink,"incompleteMethod.append("", "");..."
12,70,CONDITIONAL_EXPRESSION_DELETION_MUTATION,/Users/alex.wu/IdeaProjects/flink/flink-rpc/fl...,64,"if (toString == null) {\n toString = ""Local...","toString = ""LocalRpcInvocation("" + RpcInvocati...",@Override\n public String toString() {\n,flink,\n return toString;\n }\n
...,...,...,...,...,...,...,...,...,...
40939,57,LOOP_CONTROL_DELETION_MUTATION,/Users/alex.wu/IdeaProjects/flink/flink-runtim...,57,break;,,public static void redirectSystemOutAndErr...,flink,case DEFAULT:\n default...
40942,177,RETURN_EXPRESSION_MUTATION,/Users/alex.wu/IdeaProjects/flink/flink-runtim...,177,return LINE_SEPARATOR_BYTES[0] == buf[count - 1];,return LINE_SEPARATOR_BYTES[0] < buf[count - 1];,private synchronized boolean isLineEnd...,flink,}\n\n for (int i = 0; i...
40943,42,RETURN_EXPRESSION_MUTATION,/Users/alex.wu/IdeaProjects/flink/flink-runtim...,42,"return ""ExecutionDeploymentReport{"" + ""executi...","return ""ExecutionDeploymentReport{"" - ""executi...",@Override\n public String toString() {\n,flink,}\n
40946,127,RETURN_EXPRESSION_MUTATION,/Users/alex.wu/IdeaProjects/flink/flink-runtim...,127,return connection != null;,return connection == null;,@Override\n public boolean isCo...,flink,}\n


In [45]:
# 假设 final_df_no_chain_calls 是您的 DataFrame

# 统计每种 mutationStrategy 的出现次数
strategy_counts = final_df_no_chain_calls['mutationStrategy'].value_counts()

# 计算总样本数
total_count = len(final_df_no_chain_calls)

# 打印每种策略及其占比
print("Mutation Strategy 占比:")
for strategy, count in strategy_counts.items():
    proportion = (count / total_count) * 100
    print(f"{strategy}: {count}, 占比: {proportion:.2f}%")

Mutation Strategy 占比:
RETURN_EXPRESSION_MUTATION: 2574, 占比: 20.98%
RANDOM_BLOCK_STATEMENT_INFILL_MUTATION: 2463, 占比: 20.07%
CONDITIONAL_EXPRESSION_DELETION_MUTATION: 1849, 占比: 15.07%
COMPLETE_CONDITIONAL_BRANCH_DELETION_MUTATION: 1805, 占比: 14.71%
RELATIONAL_EXPRESSION_MUTATION: 1483, 占比: 12.09%
RANDOM_BLOCK_STATEMENT_DELETION_MUTATION: 1013, 占比: 8.26%
RANDOM_CONDITION_RESERVATION_MUTATION: 737, 占比: 6.01%
LOOP_CONTROL_DELETION_MUTATION: 232, 占比: 1.89%
SWITCH_RANDOM_BRANCH_DELETION_MUTATION: 113, 占比: 0.92%


In [46]:
# 假设 final_df_no_chain_calls 是您的 DataFrame
output_file_path = '/Users/alex.wu/java_projects/java-code-parser/mutation_infos_processed.jsonl'

# 将 DataFrame 转换为 JSON Lines 格式的字符串
jsonl_string = final_df_no_chain_calls.to_json(orient='records', lines=True)

# 将字符串写入文件
with open(output_file_path, 'w') as file:
    file.write(jsonl_string)

print(f"JSONL 文件已保存到 {output_file_path}")


JSONL 文件已保存到 /Users/alex.wu/java_projects/java-code-parser/mutation_infos_processed.jsonl
